In [71]:
%matplotlib
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import numpy as np
import tensorflow as tf
from keras.callbacks import ModelCheckpoint

Using matplotlib backend: Qt5Agg


# Step 1: Preprocess the data

In [2]:
df = pd.read_csv('Resources/charity_data.csv')
df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
df2=df.drop(['EIN', 'NAME'], axis = 1)
df2

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [4]:
df2.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [5]:
df2.groupby(df2['APPLICATION_TYPE']).count().sort_values('AFFILIATION', ascending=False)

,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
APPLICATION_TYPE,,,,,,,,,
T3,27037,27037,27037,27037,27037,27037,27037,27037,27037
T4,1542,1542,1542,1542,1542,1542,1542,1542,1542
T6,1216,1216,1216,1216,1216,1216,1216,1216,1216
T5,1173,1173,1173,1173,1173,1173,1173,1173,1173
T19,1065,1065,1065,1065,1065,1065,1065,1065,1065
T8,737,737,737,737,737,737,737,737,737
T7,725,725,725,725,725,725,725,725,725
T10,528,528,528,528,528,528,528,528,528
T9,156,156,156,156,156,156,156,156,156


In [6]:
top8=df['APPLICATION_TYPE'].value_counts()[:8].index
df2.loc[~df2['APPLICATION_TYPE'].isin(top8), 'APPLICATION_TYPE'] = 'Other'
df2.nunique()

APPLICATION_TYPE             9
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [7]:
df2.groupby(df2['APPLICATION_TYPE']).count().sort_values('AFFILIATION', ascending=False)

,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
APPLICATION_TYPE,,,,,,,,,
T3,27037,27037,27037,27037,27037,27037,27037,27037,27037
T4,1542,1542,1542,1542,1542,1542,1542,1542,1542
T6,1216,1216,1216,1216,1216,1216,1216,1216,1216
T5,1173,1173,1173,1173,1173,1173,1173,1173,1173
T19,1065,1065,1065,1065,1065,1065,1065,1065,1065
T8,737,737,737,737,737,737,737,737,737
T7,725,725,725,725,725,725,725,725,725
T10,528,528,528,528,528,528,528,528,528
Other,276,276,276,276,276,276,276,276,276


In [8]:
df2.groupby(df2['CLASSIFICATION']).count().sort_values('AFFILIATION', ascending=False).head(20)

,APPLICATION_TYPE,AFFILIATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
CLASSIFICATION,,,,,,,,,
C1000,17326,17326,17326,17326,17326,17326,17326,17326,17326
C2000,6074,6074,6074,6074,6074,6074,6074,6074,6074
C1200,4837,4837,4837,4837,4837,4837,4837,4837,4837
C3000,1918,1918,1918,1918,1918,1918,1918,1918,1918
C2100,1883,1883,1883,1883,1883,1883,1883,1883,1883
C7000,777,777,777,777,777,777,777,777,777
C1700,287,287,287,287,287,287,287,287,287
C4000,194,194,194,194,194,194,194,194,194
C5000,116,116,116,116,116,116,116,116,116


In [9]:
top6=df['CLASSIFICATION'].value_counts()[:6].index
df2.loc[~df2['CLASSIFICATION'].isin(top6), 'CLASSIFICATION'] = 'Other'
df2.nunique()

APPLICATION_TYPE             9
AFFILIATION                  6
CLASSIFICATION               7
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [10]:
df3 = pd.get_dummies(df2)

In [11]:
df3.head(100)

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
96,1,5000,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
97,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
98,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [45]:
# Remove successful target from features data
y = df3.IS_SUCCESSFUL.values
X = df3.drop(columns = "IS_SUCCESSFUL").values

# Step 2: Compile, Train, and Evaluate the Model

In [46]:
# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# Preprocess numerical data for neural network
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [59]:
# Define the deep learning model 
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=16, activation="relu", input_dim=44))
nn_model.add(tf.keras.layers.Dense(units=16, activation="relu")) 
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [60]:
nn_model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_29 (Dense)             (None, 16)                720       
_________________________________________________________________
dense_30 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_31 (Dense)             (None, 1)                 17        
Total params: 1,009
Trainable params: 1,009
Non-trainable params: 0
_________________________________________________________________


In [78]:
# Create the callback that saves the model's weights every 5 epochs
checkpoint = ModelCheckpoint("AlphabetSoupCharity.h5", encoding='utf-8', monitor='loss', verbose=1,
    save_best_only=True, mode='auto', period=5)

# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50, validation_data=(X_test, y_test), callbacks=[checkpoint]) 

Epoch 1/50
804/804 [==============================] - 1s 690us/step - loss: 0.5354 - accuracy: 0.7402 - val_loss: 41071.4727 - val_accuracy: 0.5324
Epoch 2/50
804/804 [==============================] - 1s 689us/step - loss: 0.5355 - accuracy: 0.7408 - val_loss: 57287.4492 - val_accuracy: 0.5324
Epoch 3/50
804/804 [==============================] - 1s 675us/step - loss: 0.5355 - accuracy: 0.7409 - val_loss: 42207.0195 - val_accuracy: 0.5324
Epoch 4/50
804/804 [==============================] - 1s 671us/step - loss: 0.5354 - accuracy: 0.7410 - val_loss: 55816.0156 - val_accuracy: 0.5324
Epoch 5/50
804/804 [==============================] - 1s 680us/step - loss: 0.5357 - accuracy: 0.7400 - val_loss: 40843.5352 - val_accuracy: 0.5324

Epoch 00005: loss improved from inf to 0.53568, saving model to AlphabetSoupCharity.h5
Epoch 6/50
804/804 [==============================] - 1s 677us/step - loss: 0.5352 - accuracy: 0.7408 - val_loss: 23796.5137 - val_accuracy: 0.5324
Epoch 7/50
804/804 [====

804/804 [==============================] - 1s 680us/step - loss: 0.5345 - accuracy: 0.7409 - val_loss: 73611.7109 - val_accuracy: 0.5324

Epoch 00050: loss improved from 0.53479 to 0.53449, saving model to AlphabetSoupCharity.h5


In [79]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5538 - accuracy: 0.7251
Loss: 0.5538477897644043, Accuracy: 0.7251312136650085
